In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score, learning_curve, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score, precision_score, f1_score, recall_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.inspection import permutation_importance
from math import sqrt
import json
import os
import time
from scipy import stats

np.random.seed(42)

def load_data():
    df = pd.read_csv('')

    
    feature_names = ['W', 'WW', 'H', 'D', 'DH']
    X_df = df[feature_names]

    y_series = df['MR']

    return X_df, y_series, df  

X, y, full_df = load_data()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X.columns)


expanded_param_grid = {
    'n_estimators': [50, 100, 200, 300, 500, 750, 1000, 1500, 2000],  
    'max_depth': [2, 3, 4, 5, 6, 7, 10, 15, 20, 25, 30],  
    'learning_rate': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3, 0.5, 0.7],  
    'subsample': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  
    'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  
    'min_child_weight': [1, 2, 3, 5, 7, 10, 15, 20],  
    'gamma': [0, 0.01, 0.1, 0.3, 0.5, 1.0, 2.0, 5.0],  
    'alpha': [0, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0],  
    'lambda': [0, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0]  
}


random_search = RandomizedSearchCV(
    estimator=XGBRegressor(random_state=42),
    param_distributions=expanded_param_grid,
    n_iter=50,  
    cv=5,  
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=2,
    random_state=42,
    return_train_score=True
)
random_search.fit(X_train_scaled_df, y_train)


best_params = random_search.best_params_


def create_focused_param_grid(best_params):
    focused_param_grid = {}
    for param, value in best_params.items():
        if param == 'n_estimators':
            focused_param_grid[param] = [max(50, value-200), value, min(2000, value+200)]
        elif param == 'max_depth':
            focused_param_grid[param] = [max(3, value-3), value, min(25, value+3)]
        elif param == 'learning_rate':
            focused_param_grid[param] = [max(0.0001, value/2), value, min(1.0, value*2)]
        elif param in ['subsample', 'colsample_bytree']:
            focused_param_grid[param] = [max(0.4, value-0.2), value, min(1.0, value+0.2)]
        elif param in ['min_child_weight', 'gamma', 'alpha', 'lambda']:
            focused_param_grid[param] = [max(0, value-0.5), value, min(10, value+0.5)]
        else:
            focused_param_grid[param] = [value]
    return focused_param_grid

focused_param_grid = create_focused_param_grid(best_params)


grid_search = GridSearchCV(
    estimator=XGBRegressor(random_state=42),
    param_grid=focused_param_grid,
    cv=5,
    scoring=['neg_root_mean_squared_error', 'r2', 'neg_mean_absolute_error'],
    refit='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=2,
    return_train_score=True
)
grid_search.fit(X_train_scaled_df, y_train)


best_model = grid_search.best_estimator_


y_pred = best_model.predict(X_test_scaled_df)
y_train_pred = best_model.predict(X_train_scaled_df)


print("\n== Best Parameters ==")
for param, value in best_model.get_params().items():
    print(f"{param}: {value}")


def binarize_outputs(actual, predicted, threshold=None):
    if threshold is None:
        threshold = np.median(actual)
    actual_binary = (actual > threshold).astype(int)
    predicted_binary = (predicted > threshold).astype(int)
    return actual_binary, predicted_binary, threshold

y_train_binary, y_train_pred_binary, train_threshold = binarize_outputs(y_train, y_train_pred)
y_test_binary, y_test_pred_binary, test_threshold = binarize_outputs(y_test, y_pred)


errors = y_test - y_pred
relative_errors = np.abs((y_test - y_pred) / y_test * 100)


rmse = sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

train_rmse = sqrt(mean_squared_error(y_train, y_train_pred))
train_r2 = r2_score(y_train, y_train_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)


train_accuracy = accuracy_score(y_train_binary, y_train_pred_binary)
train_precision = precision_score(y_train_binary, y_train_pred_binary)
train_recall = recall_score(y_train_binary, y_train_pred_binary)
train_f1 = f1_score(y_train_binary, y_train_pred_binary)

test_accuracy = accuracy_score(y_test_binary, y_test_pred_binary)
test_precision = precision_score(y_test_binary, y_test_pred_binary)
test_recall = recall_score(y_test_binary, y_test_pred_binary)
test_f1 = f1_score(y_test_binary, y_test_pred_binary)


kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores_rmse = -cross_val_score(best_model, X_train_scaled_df, y_train, scoring='neg_root_mean_squared_error', cv=kf)
cv_scores_r2 = cross_val_score(best_model, X_train_scaled_df, y_train, scoring='r2', cv=kf)
cv_scores_mae = -cross_val_score(best_model, X_train_scaled_df, y_train, scoring='neg_mean_absolute_error', cv=kf)


train_sizes, train_scores, test_scores = learning_curve(
    best_model,
    X_train_scaled_df,
    y_train,
    train_sizes=np.linspace(0.2, 1.0, 5),
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    shuffle=True,
    random_state=42
)


default_importance = best_model.feature_importances_
default_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Default_Importance': default_importance
}).sort_values('Default_Importance', ascending=False)


result = permutation_importance(
    best_model, X_test_scaled_df, y_test,
    n_repeats=5,
    random_state=42, n_jobs=-1
)
permutation_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Permutation_Importance_Mean': result.importances_mean,
    'Permutation_Importance_Std': result.importances_std
}).sort_values('Permutation_Importance_Mean', ascending=False)

def analyze_drug_prediction_errors(X_test, y_test, y_pred, full_df, error_threshold=15):
    
    absolute_errors = np.abs(y_test - y_pred)
    relative_errors = np.abs((y_test - y_pred) / y_test * 100)

  
    error_df = pd.DataFrame({
        'Drug': full_df.loc[y_test.index, 'Drugs'],  
        'Actual': y_test,
        'Predicted': y_pred,
        'Absolute_Error': absolute_errors,
        'Relative_Error_Percent': relative_errors
    })

    
    high_error_drugs = error_df[error_df['Relative_Error_Percent'] > error_threshold]

    
    return high_error_drugs.sort_values('Relative_Error_Percent', ascending=False)

print("\n======= Performance Metrics =======")
print("\n== Regression Metrics ==")
print(f"Training RMSE: {train_rmse:.4f}")
print(f"Testing RMSE: {rmse:.4f}")
print(f"Training R²: {train_r2:.4f}")
print(f"Testing R²: {r2:.4f}")
print(f"Training MAE: {train_mae:.4f}")
print(f"Testing MAE: {mae:.4f}")

print("\n== Classification Metrics (Binarized) ==")
print(f"Binarization threshold: {test_threshold:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Testing Accuracy: {test_accuracy:.4f}")
print(f"Training Precision: {train_precision:.4f}")
print(f"Testing Precision: {test_precision:.4f}")
print(f"Training Recall: {train_recall:.4f}")
print(f"Testing Recall: {test_recall:.4f}")
print(f"Training F1 Score: {train_f1:.4f}")
print(f"Testing F1 Score: {test_f1:.4f}")

print("\n== 5-Fold Cross-Validation Results ==")
print(f"CV RMSE: {cv_scores_rmse.mean():.4f} (±{cv_scores_rmse.std():.4f})")
print(f"CV R²: {cv_scores_r2.mean():.4f} (±{cv_scores_r2.std():.4f})")
print(f"CV MAE: {cv_scores_mae.mean():.4f} (±{cv_scores_mae.std():.4f})")

print("\n== Feature Importance (Combined Methods) ==")
print(default_importance_df)


high_error_drugs = analyze_drug_prediction_errors(
    X_test_scaled_df,
    y_test,
    y_pred,
    full_df,  
    error_threshold=15  
)

print("\n== Drugs with Highest Prediction Errors ==")
print(high_error_drugs)

plt.figure(figsize=(7, 7), dpi=100)


plt.scatter(y_test, y_pred,
           alpha=0.6,            
           color='red',         
           marker='o',           
           s=50,                 
           edgecolor='black',    
           linewidth=0.3)        


plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()],
        color='black',           
        linestyle='-',        
        linewidth=2)          

plt.title("", fontsize=14)
plt.xlabel("Actual", fontsize=12)
plt.ylabel("Predicted", fontsize=12)
plt.grid(False)
plt.tight_layout()
plt.show()


plt.figure(figsize=(7, 7), dpi=100)


plt.scatter(y_pred, errors,
           alpha=0.6,
           color='#3498db',      
           marker='o',           
           s=50,                 
           edgecolor='white',    
           linewidth=0.3)        

plt.axhline(y=0,
           color='#e74c3c',      
           linestyle='--',       
           linewidth=2)         

plt.title("", fontsize=14)
plt.xlabel("Predicted", fontsize=12)
plt.ylabel("Residuals", fontsize=12)
plt.grid(False)
plt.tight_layout()
plt.show()

